In [5]:
import pandas as pd
import requests
from io import BytesIO
import zipfile
import os

# Informações os dados a serem baixados

In [6]:
fontes = [
    {
        "Indicador": "Trade Policy Uncertainty",
        "Link": "https://www.matteoiacoviello.com/tpu_files/tpu_web_latest.xlsx",
        "Excel_sheet": "TPU_MONTHLY",
        "Column_excel_sheet": ["A","E"]
    },
    {
        "Indicador": "World Uncertainty Index",
        "Link": "https://worlduncertaintyindex.com/wp-content/uploads/2025/05/WUI_M_dataset_2025_04.xlsx",
        "Excel_sheet": "F1",
        "Column_excel_sheet": ["A", "B"]
    },
    {
        "Indicador": "Global Economic Policy Uncertainty Index",
        "Link": "https://www.policyuncertainty.com/media/Global_Policy_Uncertainty_Data.xlsx",
        "Excel_sheet": "Sheet1",
        "Column_excel_sheet": ["A", "B", "D"]
    }
]

# Downloading files

In [7]:
def download_all_links(fontes):
    os.makedirs("00.data", exist_ok=True)
    for fonte in fontes:
        try:
            resp = requests.get(fonte["Link"], verify=False)
            filename = os.path.join("00.data", fonte["Link"].split("/")[-1])
            with open(filename, "wb") as f:
                f.write(resp.content)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Failed to download {fonte['Indicador']}: {e}")

download_all_links(fontes)

c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.matteoiacoviello.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: 00.data\tpu_web_latest.xlsx


c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worlduncertaintyindex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: 00.data\WUI_M_dataset_2025_04.xlsx
Downloaded: 00.data\Global_Policy_Uncertainty_Data.xlsx


c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.policyuncertainty.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Leitura de cada arquivo Excel separadamente e salvando em dataframes distintos

In [ ]:

# Trade Policy Uncertainty
trade_policy_uncertainty_df = pd.read_excel(
    "00.data/TPU_web_latest.xlsx",
    sheet_name="TPU_MONTHLY",
    usecols="A,E",
    engine="openpyxl"
)
trade_policy_uncertainty_df.columns = ["Date", "Trade Policy Uncertainty"]
trade_policy_uncertainty_df["Date"] = pd.to_datetime(trade_policy_uncertainty_df["Date"], errors="coerce")
trade_policy_uncertainty_df = trade_policy_uncertainty_df.dropna(subset=["Date"]).set_index("Date")


In [ ]:

# World Uncertainty Index
world_uncertainty_index_df = pd.read_excel(
    "00.data/WUI_M_dataset_2025_04.xlsx",
    sheet_name="F1",
    usecols="A,B",
    engine="openpyxl"
)
world_uncertainty_index_df.columns = ["Date", "World Uncertainty Index"]
world_uncertainty_index_df["Date"] = pd.to_datetime(world_uncertainty_index_df["Date"], errors="coerce")
world_uncertainty_index_df = world_uncertainty_index_df.dropna(subset=["Date"]).set_index("Date")


In [ ]:

# Global Economic Policy Uncertainty Index
global_economic_policy_uncertainty_df = pd.read_excel(
    "00.data/Global_Policy_Uncertainty_Data.xlsx",
    sheet_name="Sheet1",
    usecols="A,B,D",
    engine="openpyxl"
)
global_economic_policy_uncertainty_df.columns = [
    "year",
    "month",
    "Global Economic Policy Uncertainty Index"
]
# Convert year and month to datetime
global_economic_policy_uncertainty_df["Date"] = pd.to_datetime(
    global_economic_policy_uncertainty_df["year"].astype(str) + "-" +
    global_economic_policy_uncertainty_df["month"].fillna(1).astype(int).astype(str) + "-01",
    errors="coerce"
)

# Merge dataframes

In [ ]:
merged_df = global_economic_policy_uncertainty_df.set_index("Date")[
    ["Global Economic Policy Uncertainty Index"]
].join([
    world_uncertainty_index_df,
    trade_policy_uncertainty_df
], how="outer")

In [26]:
merged_df.to_excel("00.data/merged_uncertainty_indices.xlsx")